In [3]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_pendulum import get_pendulum_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
tf.compat.v1.disable_eager_execution()


# Generate data

In [4]:
training_data = get_pendulum_data(100)
validation_data = get_pendulum_data(10)

# Set up model and training parameters

In [5]:
params = {}

params['input_dim'] = training_data['x'].shape[-1]
params['latent_dim'] = 1
params['model_order'] = 2
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(2*params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_x'] = 5e-4
params['loss_weight_sindy_z'] = 5e-5
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
params['widths'] = [128,64,32]

# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1000
params['learning_rate'] = 1e-4

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [6]:
num_experiments = 10
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'pendulum_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0

TRAINING
Epoch 0
   training loss 0.02408435009419918, (0.008249346, 0.00066396705, 31.650017, 0.996192)
   validation loss 0.01657882332801819, (0.008133821, 0.0004216345, 16.870039, 0.996192)
decoder loss ratio: 0.969723, decoder SINDy loss  ratio: 1.000000
Epoch 100
   training loss 0.023528359830379486, (0.007698235, 5.9400274e-05, 31.650017, 0.51132095)
   validation loss 0.015992337837815285, (0.007552202, 4.2785105e-05, 16.870039, 0.51132095)
decoder loss ratio: 0.900382, decoder SINDy loss  ratio: 1.000000
Epoch 200
   training loss 0.023552976548671722, (0.0077274214, 0.002265967, 31.650017, 0.043185737)
   validation loss 0.015969902276992798, (0.0075343796, 0.0014201283, 16.870039, 0.043185737)
decoder loss ratio: 0.898257, decoder SINDy loss  ratio: 1.000000
Epoch 300
   training loss 0.02356213703751564, (0.007736083, 0.020096159, 31.650017, 0.0041611902)
   validation loss 0.015966711565852165, (0.0075310552, 0.011899745, 16.870039, 0.0041611902)
decoder los

AttributeError: 'DataFrame' object has no attribute 'append'